In [2]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')

In [3]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)
    
print('gen_inv_last_ans_time_data...')

gen_inv_last_ans_time_data...


In [4]:
inv = pd.read_pickle(os.path.join(DATA_PATH, 'inv_time.pkl'))
ans = pd.read_pickle(os.path.join(DATA_PATH, 'ans_time.pkl'))

In [5]:
inv.columns = ['id', 'uid','time']
ans.columns = ['uid', 'time']

In [6]:
tmp = pd.concat([inv, ans])
tmp.sort_values(by=['uid', 'time'], inplace=True)
tmp = tmp[['uid', 'id', 'time']]

In [7]:
last_id = np.nan
last_author = '-1'
ts = []
t = np.nan
for author, id, ti in tqdm(tmp.values):
    if author != last_author:
        if pd.isna(id):
            t = ti
        else:
            t = np.nan
        ts.append(np.nan)
    elif pd.isna(id):
        t = ti
        ts.append(np.nan)
    else:
        ts.append(t)
    last_author = author

100%|██████████| 15144580/15144580 [00:35<00:00, 425054.95it/s]


In [8]:
tmp['last_time'] = ts
tmp = tmp[~tmp['id'].isnull()]
tmp.sort_values(by='id', inplace=True)
tmp['user_time'] = tmp['uid'].astype(str) + '_' + tmp['last_time'].fillna(-1).astype(int).astype(str)
print('NAN ratio: %f' % (tmp['last_time'].isnull().sum() / len(tmp)))

NAN ratio: 0.295646


In [16]:
tmp = tmp.reset_index(drop=True)
tmp.to_pickle(os.path.join(DATA_PATH, 'inv_last_answer_time.pkl'))

In [17]:
tmp['invite_answer_gap'] = tmp['time'] - tmp['last_time'] 
tmp[['invite_answer_gap']].to_pickle(os.path.join(SAVE_PATH, 'inv_ans_gap.pkl'))

In [18]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 262
